In [1]:
# 使用不同的max_depth对决策树进行训练

In [2]:
# 用project3中的预测用户LTV模型来实验
import numpy as np
import pandas as pd

df_sales = pd.read_csv('易速鲜花订单记录.csv')
#数据清洗===================
# 删除重复的行
df_sales = df_sales.drop_duplicates()
# 清洗掉数量小于0的数据行
df_sales = df_sales.loc[df_sales['数量'] > 0]
#总价
df_sales['总价'] = df_sales['数量'] * df_sales['单价']
df_sales.head()

# 求出 R F M
df_user = pd.DataFrame(df_sales['用户码'].unique()) #生成以用户码为主键的结构df_user
df_user.columns = ['用户码'] #设定字段名
df_user = df_user.sort_values(by='用户码',ascending=True).reset_index(drop=True) #按用户码排序
df_user #显示df_user


df_sales['消费日期'] = pd.to_datetime(df_sales['消费日期']) #转化日期格式
df_recent_buy = df_sales.groupby('用户码').消费日期.max().reset_index() #构建消费日期信息
df_recent_buy.columns = ['用户码','最近日期'] #设定字段名
df_recent_buy['R值'] = (df_recent_buy['最近日期'].max() - df_recent_buy['最近日期']).dt.days #计算最新日期与上次消费日期的天数
df_user = pd.merge(df_user, df_recent_buy[['用户码','R值']], on='用户码') #把上次消费距最新日期的天数（R值）合并至df_user结构
df_user.head() #显示df_user头几行数据

df_frequency = df_sales.groupby('用户码').消费日期.count().reset_index() #计算每个用户消费次数，构建df_frequency对象
df_frequency.columns = ['用户码','F值'] #设定字段名称
df_user = pd.merge(df_user, df_frequency, on='用户码') #把消费频率整合至df_user结构
df_user.head() #显示头几行数据

df_revenue = df_sales.groupby('用户码').总价.sum().reset_index() #根据消费总额，构建df_revenue对象
df_revenue.columns = ['用户码','M值'] #设定字段名称
df_user = pd.merge(df_user, df_revenue, on='用户码') #把消费金额整合至df_user结构
df_user.head() #显示头几行数据


#===========================

print('日期范围： %s ~ %s' % (df_sales['消费日期'].min(), df_sales['消费日期'].max()))
df_sales['消费日期'] = pd.to_datetime(df_sales['消费日期'])
print('日期范围： %s ~ %s' % (df_sales['消费日期'].min(), df_sales['消费日期'].max()))

# 求整年的 LTV 所以 把不完整的 2021年6月数据删除 时间跨度：2020 年 6 月到 2021 年 6 月 9 号
df_sales = df_sales.loc[df_sales['消费日期'] < '2021-06-01']
print('日期范围： %s ~ %s' % (df_sales['消费日期'].min(), df_sales['消费日期'].max()))

# 构建特征和标签
# 构建仅含头三个月的数据
df_sales_3m = df_sales[(df_sales.消费日期 > '2020-06-01') & (df_sales.消费日期 <= '2020-08-30')]
# 索引重置
df_sales_3m.reset_index(drop=True)

df_user_LTV = pd.DataFrame(df_sales['用户码'].unique()) #生成以用户码为主键的结构
df_user_LTV.columns = ['用户码'] #设定字段名
df_user_LTV.head() #显示头几行数据
df_R_value = df_sales_3m.groupby('用户码').消费日期.max().reset_index() #找到每个用户的最近消费日期，构建df_R_value对象
df_R_value.columns = ['用户码','最近购买日期'] #设定字段名
df_R_value['R值'] = (df_R_value['最近购买日期'].max() - df_R_value['最近购买日期']).dt.days #计算最新日期与上次消费日期的天数
df_user_LTV = pd.merge(df_user_LTV, df_R_value[['用户码','R值']], on='用户码') #把上次消费距最新日期的天数（R值）合并至df_user结构
df_F_value = df_sales_3m.groupby('用户码').消费日期.count().reset_index() #计算每个用户消费次数，构建df_F_value对象
df_F_value.columns = ['用户码','F值'] #设定字段名
df_user_LTV = pd.merge(df_user_LTV, df_F_value[['用户码','F值']], on='用户码') #把消费频率(F值)整合至df_user结构
df_M_value = df_sales_3m.groupby('用户码').总价.sum().reset_index() #计算每个用户三个月消费总额，构建df_M_value对象
df_M_value.columns = ['用户码','M值'] #设定字段名
df_user_LTV = pd.merge(df_user_LTV, df_M_value, on='用户码') #把消费总额整合至df_user结构
df_user_LTV #显示用户表结构

# 现根据数据算出每个用户据的LTV值 12个月的消费总额
# 构建每个用户整年消费总额
df_user_ly = df_sales.groupby('用户码')['总价'].sum().reset_index()
df_user_ly.columns = ['用户码', '年度LTV']
df_user_ly.head()
#构建整体LTV训练数据集
df_LTV = pd.merge(df_user_LTV, df_user_ly, on='用户码', how='left')
df_LTV


日期范围： 2020-06-01 09:09:00 ~ 2021-06-09 12:31:00
日期范围： 2020-06-01 09:09:00 ~ 2021-06-09 12:31:00
日期范围： 2020-06-01 09:09:00 ~ 2021-05-31 17:37:00


,用户码,R值,F值,M值,年度LTV
0,15100,49,3,876.00,876.00
1,15291,41,33,1372.19,4668.30
2,14688,6,82,1491.37,4949.83
3,15311,5,693,12859.50,59457.10
4,15862,89,64,354.23,659.73
...,...,...,...,...,...
356,15951,1,22,375.17,375.17
357,14745,1,7,240.60,1220.26
358,15724,0,5,103.65,218.25
359,15874,0,5,584.35,4405.88


In [3]:
# 数据集
# 创建特征集和标签集 特征集的X大写 标签集的y小写 （惯例）
X = df_LTV.drop(['用户码','年度LTV'], axis=1)
X.head()

y = df_LTV['年度LTV']
y.head()

from sklearn.model_selection import train_test_split
# 先拆分训练集和其它集
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.7,random_state = 36)
# 再把其它集拆分成验证集和测试集 
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5,random_state = 36)

In [4]:
# 导入决策树模型
from sklearn.tree import DecisionTreeRegressor
#创建默认参数的决策树模型 深度不加限定
model_dtr = DecisionTreeRegressor()
#创建深度为3的决策树模型
model_dtr_cut = DecisionTreeRegressor(max_depth=3)
model_dtr.fit(X_train, y_train)
model_dtr_cut.fit(X_train, y_train)

y_valid_preds_dtr = model_dtr.predict(X_valid)
y_valid_preds_dtr_cut = model_dtr_cut.predict(X_valid)

from sklearn.metrics import r2_score, median_absolute_error
print('训练集上的R方分数-决策树： %0.4f' % r2_score(y_train, model_dtr.predict(X_train)))
print('训练集上的R平方分数-深度为3的决策树: %0.4f' % r2_score(y_train, model_dtr_cut.predict(X_train)))
print('测试集上的R平方分数-决策树: %0.4f' % r2_score(y_valid, model_dtr.predict(X_valid)))
print('测试集上的R平方分数-深度为3的决策树: %0.4f' % r2_score(y_valid, model_dtr_cut.predict(X_valid)))

训练集上的R方分数-决策树： 1.0000
训练集上的R平方分数-深度为3的决策树: 0.9056
测试集上的R平方分数-决策树: 0.2086
测试集上的R平方分数-深度为3的决策树: 0.2898


In [5]:
# 添加线性回归模型的正则项

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
#创建线性回归模型
model_lr = LinearRegression() 
# 创建Lasso模型
model_lasso = Lasso()
# 创建Ridge模型
model_ridge = Ridge()
#三个拟合
model_lr.fit(X_train, y_train)
model_lasso.fit(X_train, y_train)
model_ridge.fit(X_train, y_train)

#对比三个模型的r方
from sklearn.metrics import r2_score, median_absolute_error 
#导入Sklearn评估模块
print('训练集上的R平方分数-线性回归: %0.4f' % r2_score(y_train, model_lr.predict(X_train)))
print('训练集上的R平方分数-Lasso回归: %0.4f' % r2_score(y_train, model_lasso.predict(X_train)))
print('训练集上的R平方分数-Ridge回归: %0.4f' % r2_score(y_train, model_ridge.predict(X_train)))
print('测试集上的R平方分数-线性回归: %0.4f' % r2_score(y_valid, model_lr.predict(X_valid)))
print('测试集上的R平方分数-Lasso回归: %0.4f' % r2_score(y_valid, model_lasso.predict(X_valid)))
print('测试集上的R平方分数-Ridge回归: %0.4f' % r2_score(y_valid, model_ridge.predict(X_valid)))

训练集上的R平方分数-线性回归: 0.7560
训练集上的R平方分数-Lasso回归: 0.7560
训练集上的R平方分数-Ridge回归: 0.7560
测试集上的R平方分数-线性回归: 0.2983
测试集上的R平方分数-Lasso回归: 0.2983
测试集上的R平方分数-Ridge回归: 0.2983
